In [83]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [84]:
baseline_clean = pd.read_pickle('data_clean/baseline_clean.pkl')
survey_clean = pd.read_pickle('data_clean/survey_clean.pkl')
follow_survey = pd.read_pickle('data_clean/follow_survey.pkl')

In [85]:
itit_df = pd.merge(baseline_clean, survey_clean, on=['trip_id', 'user_id'], how='left')

In [86]:
# correct_travel_date_year: Adjusts the year in travel_date to match the year of baseline_date if it falls outside the range [2022, 2025]
itit_df['travel_date'] = itit_df.apply(lambda row: row['travel_date'].replace(year=row['baseline_date'].year) if row['travel_date'].year < 2022 or row['travel_date'].year > 2025 else row['travel_date'], axis=1)

In [87]:
# Add missing surveys
def add_missing_surveys(df):
    survey_list = []
    
    for (trip_id, user_id), group in df.groupby(['trip_id', 'user_id']):
        travel_date = group['travel_date'].iloc[0]
        duration = int(group['travel_duration'].iloc[0])
        
        # Create a DataFrame with all expected survey dates
        all_days = pd.date_range(start=travel_date, periods=duration, freq='D')
        existing_days = group['finished'].dt.date.unique()
        
        # Filter out days that already have surveys
        missing_days = [day for day in all_days if day.date() not in existing_days]
        
        # Create DataFrame for missing surveys
        if missing_days:
            missing_surveys = pd.DataFrame({
                'trip_id': trip_id,
                'user_id': user_id,
                'travel_date': travel_date,
                'finished': pd.to_datetime(missing_days),
                'travel_duration': duration
            })
            
            # Copy constant baseline variables
            baseline_vars = ['baseline_date', 'age', 'gender', 'country_iso2c', 'country_clean', 'continent_clean', 
                             'health_chronic', 'latitude', 'longitude', 'smoking_status', 'travel_purpose', 'trip_number']
            for var in baseline_vars:
                missing_surveys[var] = group[var].iloc[0]
                
            # Append missing surveys to the list
            survey_list.append(missing_surveys)
        
        # Append existing surveys to the list
        survey_list.append(group)
    
    # Concatenate all DataFrames and sort
    df_concat = pd.concat(survey_list).sort_values(by=['trip_id', 'user_id', 'finished']).reset_index(drop=True)

    # Ensure finished column is properly filled
    df_concat['finished'] = df_concat['finished'].ffill().bfill()

    # Ensure no duplicate rows by dropping the extra row caused by the duration miscalculation
    df_concat = df_concat.drop_duplicates(subset=['trip_id', 'user_id', 'finished'])
    
    return df_concat

itit_df = add_missing_surveys(itit_df)

In [88]:
# Filter out surveys that occurred before the departure date
itit_df = itit_df[itit_df['finished'] >= itit_df['travel_date']]

In [89]:
# Reassign Travel Duration Based on Trip ID Count
itit_df['travel_duration'] = itit_df.groupby('trip_id')['trip_id'].transform('count')

In [90]:
# Fixing follow up survey with allocating feedback 1 or 2 correctly
finish_trip_df = (itit_df.loc[itit_df.groupby('trip_id')['finished'].idxmax(), ['user_id', 'trip_id', 'finished']]
                      .rename(columns={'finished': 'finish_trip_date'}))
follow_survey_corr = pd.merge(
    follow_survey, 
    finish_trip_df, 
    how='left', 
    on=['trip_id', 'user_id'],
)

In [91]:
# Ensure the 'finished' and 'finish_trip_date' columns are in datetime format
follow_survey_corr['finished'] = pd.to_datetime(follow_survey_corr['finished'])
follow_survey_corr['finish_trip_date'] = pd.to_datetime(follow_survey_corr['finish_trip_date'])

# Define the function to classify follow-up surveys
def classify_follow_ups(group):
    follow_ups = []
    seen_follow_up_1 = False
    seen_follow_up_2 = False
    
    for idx, row in group.iterrows():
        days_since_trip = (row['finished'] - row['finish_trip_date']).days
        
        # Check if it's Follow-up 1 or Follow-up 2
        if 4 <= days_since_trip <= 21:  # Within 7-day window (flexible)
            if not seen_follow_up_1:
                follow_ups.append("Follow-up 1")
                seen_follow_up_1 = True
            else:
                follow_ups.append("Duplicate")
        elif 21 < days_since_trip <= 45:  # Within 30-day window (flexible)
            if not seen_follow_up_2:
                follow_ups.append("Follow-up 2")
                seen_follow_up_2 = True
            else:
                follow_ups.append("Duplicate")
        else:
            # Assign based on closest range if only one survey exists
            if not seen_follow_up_1 and not seen_follow_up_2:
                if abs(days_since_trip - 7) < abs(days_since_trip - 30):
                    follow_ups.append("Follow-up 1")
                    seen_follow_up_1 = True
                else:
                    follow_ups.append("Follow-up 2")
                    seen_follow_up_2 = True
            else:
                follow_ups.append("Duplicate")
    
    return pd.Series(follow_ups, index=group.index)

# Group by trip_id and classify follow-up surveys
follow_survey_corr = follow_survey_corr.sort_values(by=['trip_id', 'finished'])
follow_survey_corr['follow_up_survey'] = (
    follow_survey_corr.groupby('trip_id', group_keys=False).apply(classify_follow_ups)
)

/var/folders/4x/g578bp9s1bs7z_0185h2n1wr0000gn/T/ipykernel_95015/527991706.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  follow_survey_corr.groupby('trip_id', group_keys=False).apply(classify_follow_ups)


In [92]:
# Define the aggregation logic for the desired columns
def aggregate_column(column):
    # Drop NAs, keep unique values, and join them with a comma
    return ', '.join(column.dropna().astype(str).unique())

# Create a new DataFrame with aggregated values per trip_id
aggregated_values = (
    follow_survey_corr.groupby("trip_id", as_index=False)
    .agg({
        "diagnosis": aggregate_column,
        "diagnosis_date": lambda x: ', '.join(x.dropna().astype(str).unique()),
        "self_treament": aggregate_column
    })
)

# Merge the aggregated values back into the original dataset
follow_survey_corr = follow_survey_corr.merge(aggregated_values, on="trip_id", suffixes=('', '_filled'))

# Fill the original columns with the filled values
follow_survey_corr["diagnosis"] = follow_survey_corr["diagnosis_filled"]
follow_survey_corr["diagnosis_date"] = follow_survey_corr["diagnosis_date_filled"]
follow_survey_corr["self_treament"] = follow_survey_corr["self_treament_filled"]

# Drop the intermediate filled columns
follow_survey_corr = follow_survey_corr.drop(columns=["diagnosis_filled", "diagnosis_date_filled", "self_treament_filled"])

In [93]:
follow_survey_corr.to_csv('data_clean/follow_survey_corr.csv', index=False)

In [94]:
# Filter out duplicates
follow_survey_corr = follow_survey_corr[follow_survey_corr['follow_up_survey'] != "Duplicate"]

In [95]:
# Pivot the data to wide format
follow_survey_wide = follow_survey_corr.pivot(
    index=['trip_id', 'user_id'],
    columns='follow_up_survey'
)

# Flatten the multi-index columns and rename them
follow_survey_wide.columns = [
    f"{str(col[1]).replace(' ', '_').lower()}_{col[0]}" 
    for col in follow_survey_wide.columns
]

# Reset the index to make 'trip_id' and 'user_id' columns
follow_survey_wide = follow_survey_wide.reset_index()

In [96]:
itit_df_clean = pd.merge(
    itit_df, 
    follow_survey_wide, 
    how='left', 
    on=['trip_id', 'user_id'],
)

In [97]:
# Function to generate a consistent 10-character alphanumeric hash
def generate_hash(value):
    unique_value = f"{value}"  # Ensure it's a string
    hashed_value = abs(hash(unique_value))  # Get the hash and ensure it's positive
    return str(hashed_value)[:15]  # Take the first 10 characters

# Apply the hash function to the columns
itit_df_clean['trip_id'] = itit_df_clean['trip_id'].astype('category').cat.codes.apply(lambda x: generate_hash(x))
itit_df_clean['user_id'] = itit_df_clean['user_id'].astype('category').cat.codes.apply(lambda x: generate_hash(x))

In [98]:
#export csv
itit_df_clean.to_csv('data_clean/itit_df_clean.csv', index=False)

In [99]:
itit_df_clean.user_id.value_counts()

user_id
720217501014479    545
556474885282632    455
277587465624496    365
801577621909135    365
413646352847780    365
                  ... 
384399750461871      2
463925000141011      1
695902468025654      1
442562437470473      1
510394461004765      1
Name: count, Length: 1432, dtype: int64

In [80]:
itit_df_clean.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49778 entries, 0 to 49777
Data columns (total 156 columns):
 #    Column                             Dtype         
---   ------                             -----         
 0    trip_id                            object        
 1    user_id                            object        
 2    travel_date                        datetime64[ns]
 3    finished                           datetime64[ns]
 4    travel_duration                    int64         
 5    baseline_date                      datetime64[ns]
 6    age                                float64       
 7    gender                             object        
 8    country_iso2c                      object        
 9    country_clean                      object        
 10   continent_clean                    object        
 11   health_chronic                     object        
 12   latitude                           float64       
 13   longitude                          float64  

In [483]:
#only participant that filled at least one survey
itit_df_1plus_survey=itit_df.groupby('trip_id').filter(lambda x: x['nausea'].notna().any())


In [484]:
#filled missing survey based on symptoms length (forward and backward)
def fill_columns_forward(data, columns,baselevel, max_repetition=4):
    # Loop through each column provided
    for column in columns:
        # Create a new column to store the filled data
        fill_column = f'{column}_filled'
        data[fill_column] = data[column].copy()

        # Loop through each trip_id
        for tid, tdata in data.groupby('trip_id'):
            # Sort the grouped data by 'finished' for chronological order
            tdata = tdata.sort_values(by='finished')

            # We need to work with indices because we look back at previous rows
            indices = tdata.index
            for i in range(len(indices)):
                idx = indices[i]
                current_value = tdata.at[idx, column]

                if pd.isna(current_value):
                    # Look back to count repetitions of the last valid entry if it exists
                    last_valid = None
                    count_back = 0
                    for j in range(1, min(max_repetition, i) + 1):
                        back_idx = indices[i - j]
                        back_value = tdata.at[back_idx, column]
                        if not pd.isna(back_value):
                            if last_valid is None:
                                last_valid = back_value
                            if back_value == last_valid:
                                count_back += 1
                            else:
                                break

                    # If the count is less than max_repetition and last_valid is not 'none', fill with last_valid
                    if count_back < max_repetition and last_valid != baselevel[column]:
                        data.loc[idx, fill_column] = last_valid
                else:
                    # Update the filled column with current value
                    data.loc[idx, fill_column] = current_value

    return data

def fill_columns_backward(data, columns,baselevel, max_repetition=4):
    # Loop through each column provided
    for column in columns:
        fill_column = f'{column}_filled'
        
        # Ensure the 'nausea_filled' column exists, copying the original data if not
        if fill_column not in data.columns:
            data[fill_column] = data[column].copy()

        # Loop through each trip_id
        for tid, tdata in data.groupby('trip_id'):
            # Sort the grouped data by 'finished' in descending order for backward filling
            tdata = tdata.sort_values(by='finished', ascending=False)

            # We need to work with indices because we look back at previous rows
            indices = tdata.index
            last_valid = None
            count_back = 0
            continuous_na = False  # Track if we are in a continuous NA segment

            for i in range(len(indices)):
                idx = indices[i]
                current_value = tdata.at[idx, fill_column]

                if pd.isna(current_value):
                    if continuous_na:
                        # If we are in a continuous NA segment, reset last_valid
                        last_valid = None
                        count_back = 0
                    elif last_valid is not None and last_valid != baselevel[column] and count_back < max_repetition:
                        data.loc[idx, fill_column] = last_valid
                        count_back += 1
                    else:
                        # Reset the tracking variables if conditions to fill are not met
                        last_valid = None
                        count_back = 0
                    continuous_na = True  # Mark that we are in a continuous NA segment
                else:
                    # Reset continuous NA tracking as we hit a non-NA value
                    continuous_na = False
                    if current_value == last_valid:
                        count_back += 1
                    else:
                        last_valid = current_value
                        count_back = 1

    return data

columns_to_fill = ['nausea', 'vomiting', 'stomach_pain', 'diarrhea',
       'constipation','cough', 'sore_throat', 'runny_nose',
       'out_of_breath_resting', 'out_of_breath_running','rash',
       'itchy_insect_bite', 'itchy_other', 'sunburn', 'itchy_red_eyes','fever', 'dizziness', 'ear_ache', 'headache', 'pain_eyes',
       'musle_pain', 'aching_limbs','pain_joint',
       'swelling_joint', 'location_swelling',
       'body_other',
       'impact',
       'rating_day'
       ]
base_level = {
    'nausea': 'none',
    'vomiting': 'none',
    'stomach_pain': 'none',
    'diarrhea': 'none',
    'constipation': 'none',
    'cough': 'none',
    'sore_throat': 'none',
    'runny_nose': 'none',
    'out_of_breath_resting': 'none',
    'out_of_breath_running': 'none',
    'rash': 'none',
    'itchy_insect_bite': 'none',
    'itchy_other': 'none',
    'sunburn': 'none',
    'itchy_red_eyes': 'none',
    'fever': 'none',
    'dizziness': 'none',
    'ear_ache': 'none',
    'headache': 'none',
    'pain_eyes': 'none',
    'musle_pain': 'none',
    'aching_limbs': 'none',
    'pain_joint': 'none',
    'swelling_joint': 'none',
    'location_swelling': 'none',
    'body_other': 'No',
    'impact':'Did not affect my activities',
    'rating_day': 'It was a great day'
}

itit_filled_df = fill_columns_forward(itit_df_1plus_survey, columns_to_fill, baselevel=base_level)
itit_filled_df = fill_columns_backward(itit_filled_df, columns_to_fill, baselevel=base_level)

In [485]:
# Create the filled column names
filled_columns = [f'{column}_filled' for column in columns_to_fill]

# Adjust the base level mapping to match the filled column names
filled_base_level = {f'{column}_filled': base_level[column] for column in columns_to_fill}

# Fill NA values based on the adjusted base level mapping
itit_filled_df[filled_columns] = itit_filled_df[filled_columns].fillna(filled_base_level)

In [486]:
def fill_numeric_columns(data, columns, priority='last'):
    for column in columns:
        fill_column = f'{column}_filled'
        data[fill_column] = data[column].copy()

        for tid, tdata in data.groupby('trip_id'):
            tdata = tdata.sort_values(by='finished')
            indices = tdata.index.tolist()
            n = len(indices)

            I = 0
            while I < n:
                idx = indices[I]
                if pd.isna(tdata.at[idx, column]):
                    start_idx = I
                    # Find the segment of missing values
                    while I < n and pd.isna(tdata.at[indices[I], column]):
                        I += 1
                    end_idx = I

                    # Get the previous and next valid values
                    prev_value = tdata.at[indices[start_idx - 1], column] if start_idx > 0 else None
                    next_value = tdata.at[indices[end_idx], column] if end_idx < n else None

                    # Fill the missing values based on the priority
                    num_missing = end_idx - start_idx
                    half_point = (num_missing + 1) // 2

                    if prev_value is not None and next_value is not None:
                        if priority == 'last':
                            for j in range(num_missing):
                                fill_idx = indices[start_idx + j]
                                if j < half_point:
                                    data.loc[fill_idx, fill_column] = prev_value
                                else:
                                    data.loc[fill_idx, fill_column] = next_value
                        elif priority == 'first':
                            for j in range(num_missing):
                                fill_idx = indices[start_idx + j]
                                if j < half_point:
                                    data.loc[fill_idx, fill_column] = next_value
                                else:
                                    data.loc[fill_idx, fill_column] = prev_value
                    elif prev_value is not None:
                        for j in range(num_missing):
                            fill_idx = indices[start_idx + j]
                            data.loc[fill_idx, fill_column] = prev_value
                    elif next_value is not None:
                        for j in range(num_missing):
                            fill_idx = indices[start_idx + j]
                            data.loc[fill_idx, fill_column] = next_value
                else:
                    I += 1

    return data
columns_to_fill_numeric = ['trip_id',
 'user_id',
 'travel_date',
 'finished',
 'travel_duration',
 'baseline_date',
 'age',
 'gender',
 'country_iso2c',
 'country_clean',
 'continent_clean',
 'health_chronic',
 'latitude',
 'longitude',
 'smoking_status',
 'travel_purpose',
 'trip_number',
 'clouds','survey_latitude',
 'survey_longitude',
 'clouds',
 'dew_point',
 'feels_like',
 'humidity',
 'pressure',
 'rain_1h',
 'snow_1h',
 'sunrise',
 'sunset',
 'temp',
 'uvi',
 'visibility',
 'main_weather',
 'description_weather',
 'air_quality_components_co',
 'air_quality_components_nh_3',
 'air_quality_components_no',
 'air_quality_components_no_2',
 'air_quality_components_o_3',
 'air_quality_components_pm_10',
 'air_quality_components_pm_2_5',
 'air_quality_components_so_2',
 'wind_deg',
 'wind_gust',
 'wind_speed',
  'air_quality_main']
itit_filled_df = fill_numeric_columns(itit_filled_df, columns_to_fill_numeric, priority='last')

In [487]:
#remove initial columns
itit_filled_df = itit_filled_df.drop(columns=columns_to_fill)
itit_filled_df = itit_filled_df.drop(columns=columns_to_fill_numeric)
itit_filled_df = itit_filled_df.drop(columns=['respi_any','gastro_any','skin_any','body_any','joint_any'])

In [488]:
#creat dataset without gps position
filled_columns_numeric = [f'{column}_filled' for column in columns_to_fill_numeric[columns_to_fill_numeric.index('clouds'):columns_to_fill_numeric.index('air_quality_main') + 1]]
itit_nogps_filled_df = itit_filled_df.copy()
itit_nogps_filled_df = itit_nogps_filled_df.drop(columns=filled_columns_numeric)

In [489]:
#creat dataset with gps position
itit_filled_df=itit_filled_df.groupby('trip_id_filled').filter(lambda x: x['air_quality_main_filled'].notna().any())

In [490]:

itit_filled_df[['rain_1h_filled','snow_1h_filled','wind_gust_filled']]=itit_filled_df[['rain_1h_filled','snow_1h_filled','wind_gust_filled']].fillna(0)

In [491]:
itit_nogps_filled_df.to_pickle('data_clean/itit_nogps_filled_df.pkl')
itit_filled_df.to_pickle('data_clean/itit_filled_df.pkl')

In [478]:
itit_filled_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 22149 entries, 11 to 39345
Data columns (total 73 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   nausea_filled                         22149 non-null  object        
 1   vomiting_filled                       22149 non-null  object        
 2   stomach_pain_filled                   22149 non-null  object        
 3   diarrhea_filled                       22149 non-null  object        
 4   constipation_filled                   22149 non-null  object        
 5   cough_filled                          22149 non-null  object        
 6   sore_throat_filled                    22149 non-null  object        
 7   runny_nose_filled                     22149 non-null  object        
 8   out_of_breath_resting_filled          22149 non-null  object        
 9   out_of_breath_running_filled          22149 non-null  object        
 10  ra